In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
X=np.load("../input/fasttext-on-malware/bigram_fasttext_20000.npy")

In [ ]:
import pickle
with open("../input/fasttext-on-malware/tfidf_targets.pkl","rb") as f:
    Y=pickle.load(f)

In [ ]:
Y=np.array(Y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=42,test_size=0.2,stratify=Y)

In [ ]:
import optuna

In [ ]:
from sklearn.model_selection import cross_val_score
def objective(trial):
    params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'max_depth': trial.suggest_int('max_depth', 4, 50)
        }
    clf = RandomForestClassifier(random_state=42,**params)
    return cross_val_score(clf,X_train,Y_train,n_jobs=-1,cv=4).mean()

In [ ]:
study=optuna.create_study(direction='maximize',study_name="Fandom_forest_trial")
study.optimize(objective,n_trials=100)

In [ ]:
trial=study.best_trial
for key, value in trial.params.items():
    print('\t\t{}: {}'.format(key, value))

In [ ]:
clf = RandomForestClassifier(random_state=42,**trial.params)

In [ ]:
clf.fit(X_train,Y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(Y_test,clf.predict(X_test),normalize="true")
import matplotlib.pyplot as plt
import seaborn as sns
fig,ax=plt.subplots(figsize=(10,10))
sns.heatmap(cf, annot=True,cmap="Blues",linewidth=1,linecolor="black",square=True,vmax=1.0)
ax.set_xticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
ax.set_yticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
plt.xticks(rotation=45)

In [ ]:
clf.score(X_test,Y_test)

accuracy 0.907

In [ ]:
from sklearn.svm import SVC

In [ ]:
def objective(trial):
    svc_c = trial.suggest_loguniform('svc_c', 1e-10, 1e10)
    classifier_obj = SVC(C=svc_c,kernel="rbf",gamma="auto")
    return cross_val_score(classifier_obj,X_train,Y_train,n_jobs=-1,cv=4).mean()

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

In [ ]:
trial=study.best_trial
clf=SVC(C=trial.params["svc_c"],kernel="rbf",gamma="scale")
clf.fit(X_train,Y_train)

In [ ]:
cf=confusion_matrix(Y_test,clf.predict(X_test),normalize="true")
fig,ax=plt.subplots(figsize=(10,10))
sns.heatmap(cf, annot=True,cmap="Blues",linewidth=1,linecolor="black",square=True,vmax=1.0)
ax.set_xticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
ax.set_yticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
plt.xticks(rotation=45)
plt.yticks(rotation=0)

In [ ]:
clf.score(X_test,Y_test)

accuracy=0.87

In [ ]:
!pip install mlflow

In [ ]:
import mlflow
import mlflow.sklearn

In [ ]:
mlflow.set_experiment(experiment_name="MlFlow SVM")

In [ ]:
with mlflow.start_run():
    trial=study.best_trial
    clf=SVC(C=trial.params["svc_c"],kernel="rbf",gamma="scale")
    clf.fit(X_train,Y_train)
    mlflow.log_param("C",trial.params["svc_c"])
    mlflow.log_param("Gamma","scale")
    mlflow.log_param("kernel","rbf")
    mlflow.log_metric("accuracy",clf.score(X_test,Y_test))

In [ ]:
!mlflow ui

In [ ]:
uri = mlflow.tracking.get_tracking_uri()

In [ ]:
uri

In [ ]:
!mlflow ui

In [ ]:
type(clf.score(X_test,Y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf=KNeighborsClassifier(n_neighbors=20)
clf.fit(X_train,Y_train)

In [ ]:
X_train.shape

In [ ]:
cf=confusion_matrix(Y_test,clf.predict(X_test),normalize="true")
fig,ax=plt.subplots(figsize=(10,10))
sns.heatmap(cf, annot=True,cmap="Blues",linewidth=1,linecolor="black",square=True,vmax=1.0)
ax.set_xticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
ax.set_yticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
plt.xticks(rotation=45)

In [ ]:
clf.score(X_test,Y_test)

accuracy=0.8356